In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
%matplotlib inline
import librosa 
import librosa.display
import scipy 
import os
from tqdm import tqdm
import tensorflow as tf



from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,Activation,Flatten
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
import sklearn

n_fft = 2048    
n_mels = 128
hop_size = 512
Hann_window = "hann"

In [40]:
audio_nparray_path = 'nparray'
metadata = pd.read_csv('annotations1.csv')
metadata.head()

,Unnamed: 0,fname,label,split,fold
0,0,02639.npy,Bark,train,1
1,1,1580.npy,Bark,train,1
2,2,24030.npy,Bark,train,1
3,3,316499.npy,Bark,train,1
4,4,413718.npy,Bark,train,1


In [43]:
def mfcc_waveform(npfiler) :
    sample_rate =sr = 44100
    npfiler = npfiler.flatten()
    
    npfiler = npfiler / np.max(np.abs(npfiler))
    mfccs_features = librosa.feature.mfcc(y= npfiler, sr=sample_rate,n_fft = 2048, hop_length = 512,  n_mfcc=128 , window ="hann")
    
    mfccs_scaled_features = np.mean(mfccs_features,axis=1)
    mfccs_scaled_features = mfccs_scaled_features / np.max(np.abs(mfccs_scaled_features))
    return mfccs_scaled_features

In [44]:
Featured_audio = []
for index_num,row in tqdm(metadata.iterrows()):
    file_name = os.path.join(os.path.abspath(audio_nparray_path),str(row["fname"]))
    file = np.load(file_name)
    final_class_labels=row["label"]
    data=mfcc_waveform(file)
    Featured_audio.append([data,final_class_labels])

1000it [00:12, 80.04it/s]


In [45]:
### converting extracted_features to Pandas dataframe
Featured_audio_df=pd.DataFrame(Featured_audio,columns=['feature','class'])
Featured_audio_df

,feature,class
0,"[-1.0, 0.6130455, -0.05815486, 0.059190605, 0....",Bark
1,"[-1.0, 0.30551824, -0.06063275, 0.07108717, -0...",Bark
2,"[-1.0, 0.43016234, -0.11208329, 0.06396888, 0....",Bark
3,"[-1.0, 0.40302104, 0.046417512, 0.18789862, 0....",Bark
4,"[-1.0, 0.6392881, -0.08864459, 0.028237982, 0....",Bark
...,...,...
995,"[-1.0, 0.7555196, -0.082961194, 0.13166964, -0...",Walk_and_footsteps
996,"[-1.0, 0.49947548, -0.16924296, 0.05378395, -0...",Walk_and_footsteps
997,"[-1.0, 0.20934477, -0.082816415, 0.1386396, 0....",Walk_and_footsteps
998,"[-1.0, 0.3345854, -0.1770519, 0.08880131, 0.00...",Walk_and_footsteps


In [47]:
extracted_features_df = Featured_audio_df
### Split the dataset into independent and dependent dataset
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())

### Label Encoding
y=np.array(pd.get_dummies(y))
X=np.array(extracted_features_df['feature'].tolist())
y=np.array(extracted_features_df['class'].tolist())
from sklearn.preprocessing import LabelEncoder
# Convert string labels to integer labels(targets)
encoder =  LabelEncoder()
y1 = encoder.fit_transform(y)

# Convert target into one-hot encodings
Y = pd.get_dummies(y1).values



In [48]:
from sklearn.model_selection import train_test_split

# Split data into train-test split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2,random_state=0)

# from sklearn.preprocessing import StandardScaler
# sc= StandardScaler()
# X_train= sc.fit_transform(X_train) 
# X_test = sc.fit_transform(X_train) 
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)
num_labels=Y.shape[1]
model=Sequential()
model.add(Flatten(input_shape=(128,)))
###first layer
model.add(Dense(units=100, activation='relu'))
model.add(Dropout(0.5))

###second layer
model.add(Dense(units=200, activation='relu'))
model.add(Dropout(0.5))

###third layer
model.add(Dense(units=200, activation='relu'))
model.add(Dropout(0.5))

###final layer

model.add(Dense(num_labels, activation='softmax'))


In [49]:
model.compile( optimizer = 'adam' , 
              loss='mean_squared_error',
            metrics=['accuracy'] )

#stop_early = tf.keras.callbacks.EarlyStopping(monitor='accuracy', patience=30)

# Model training
model_train = model.fit(X_train, y_train, batch_size=32, epochs=500)


# Evaluate the model with test data
loss, accuracy = model.evaluate(X_test, y_test, verbose=1)



Epoch 1/500
25/25 [==============================] - 1s 3ms/step - loss: 0.0902 - accuracy: 0.0862
Epoch 2/500
25/25 [==============================] - 0s 4ms/step - loss: 0.0900 - accuracy: 0.1112
Epoch 3/500
25/25 [==============================] - 0s 4ms/step - loss: 0.0899 - accuracy: 0.1225
Epoch 4/500
25/25 [==============================] - 0s 3ms/step - loss: 0.0897 - accuracy: 0.1213
Epoch 5/500
25/25 [==============================] - 0s 4ms/step - loss: 0.0894 - accuracy: 0.1488
Epoch 6/500
25/25 [==============================] - 0s 4ms/step - loss: 0.0887 - accuracy: 0.1612
Epoch 7/500
25/25 [==============================] - 0s 4ms/step - loss: 0.0879 - accuracy: 0.1675
Epoch 8/500
25/25 [==============================] - 0s 4ms/step - loss: 0.0869 - accuracy: 0.2000
Epoch 9/500
25/25 [==============================] - 0s 4ms/step - loss: 0.0858 - accuracy: 0.2412
Epoch 10/500
25/25 [==============================] - 0s 4ms/step - loss: 0.0839 - accuracy: 0.2650
Epoch 11/

25/25 [==============================] - 0s 4ms/step - loss: 0.0386 - accuracy: 0.7387
Epoch 84/500
25/25 [==============================] - 0s 3ms/step - loss: 0.0379 - accuracy: 0.7337
Epoch 85/500
25/25 [==============================] - 0s 3ms/step - loss: 0.0389 - accuracy: 0.7275
Epoch 86/500
25/25 [==============================] - 0s 4ms/step - loss: 0.0387 - accuracy: 0.7287
Epoch 87/500
25/25 [==============================] - 0s 3ms/step - loss: 0.0382 - accuracy: 0.7138
Epoch 88/500
25/25 [==============================] - 0s 3ms/step - loss: 0.0383 - accuracy: 0.7337
Epoch 89/500
25/25 [==============================] - 0s 3ms/step - loss: 0.0368 - accuracy: 0.7375
Epoch 90/500
25/25 [==============================] - 0s 3ms/step - loss: 0.0368 - accuracy: 0.7412
Epoch 91/500
25/25 [==============================] - 0s 3ms/step - loss: 0.0380 - accuracy: 0.7312
Epoch 92/500
25/25 [==============================] - 0s 3ms/step - loss: 0.0358 - accuracy: 0.7487
Epoch 93/500


25/25 [==============================] - 0s 3ms/step - loss: 0.0265 - accuracy: 0.8188
Epoch 165/500
25/25 [==============================] - 0s 3ms/step - loss: 0.0272 - accuracy: 0.8175
Epoch 166/500
25/25 [==============================] - 0s 3ms/step - loss: 0.0262 - accuracy: 0.8300
Epoch 167/500
25/25 [==============================] - 0s 2ms/step - loss: 0.0252 - accuracy: 0.8363
Epoch 168/500
25/25 [==============================] - 0s 2ms/step - loss: 0.0264 - accuracy: 0.8238
Epoch 169/500
25/25 [==============================] - 0s 2ms/step - loss: 0.0252 - accuracy: 0.8363
Epoch 170/500
25/25 [==============================] - 0s 2ms/step - loss: 0.0260 - accuracy: 0.8325
Epoch 171/500
25/25 [==============================] - 0s 2ms/step - loss: 0.0265 - accuracy: 0.8163
Epoch 172/500
25/25 [==============================] - 0s 2ms/step - loss: 0.0238 - accuracy: 0.8413
Epoch 173/500
25/25 [==============================] - 0s 2ms/step - loss: 0.0265 - accuracy: 0.8225
Epoc

25/25 [==============================] - 0s 2ms/step - loss: 0.0199 - accuracy: 0.8700
Epoch 246/500
25/25 [==============================] - 0s 2ms/step - loss: 0.0198 - accuracy: 0.8675
Epoch 247/500
25/25 [==============================] - 0s 2ms/step - loss: 0.0233 - accuracy: 0.8450
Epoch 248/500
25/25 [==============================] - 0s 3ms/step - loss: 0.0196 - accuracy: 0.8725
Epoch 249/500
25/25 [==============================] - 0s 3ms/step - loss: 0.0204 - accuracy: 0.8750
Epoch 250/500
25/25 [==============================] - 0s 2ms/step - loss: 0.0180 - accuracy: 0.8850
Epoch 251/500
25/25 [==============================] - 0s 2ms/step - loss: 0.0189 - accuracy: 0.8775
Epoch 252/500
25/25 [==============================] - 0s 2ms/step - loss: 0.0192 - accuracy: 0.8775
Epoch 253/500
25/25 [==============================] - 0s 2ms/step - loss: 0.0192 - accuracy: 0.8737
Epoch 254/500
25/25 [==============================] - 0s 2ms/step - loss: 0.0191 - accuracy: 0.8737
Epoc

25/25 [==============================] - 0s 2ms/step - loss: 0.0149 - accuracy: 0.9075
Epoch 327/500
25/25 [==============================] - 0s 2ms/step - loss: 0.0168 - accuracy: 0.8888
Epoch 328/500
25/25 [==============================] - 0s 2ms/step - loss: 0.0174 - accuracy: 0.8825
Epoch 329/500
25/25 [==============================] - 0s 2ms/step - loss: 0.0169 - accuracy: 0.8863
Epoch 330/500
25/25 [==============================] - 0s 2ms/step - loss: 0.0161 - accuracy: 0.8963
Epoch 331/500
25/25 [==============================] - 0s 2ms/step - loss: 0.0160 - accuracy: 0.8888
Epoch 332/500
25/25 [==============================] - 0s 2ms/step - loss: 0.0159 - accuracy: 0.9000
Epoch 333/500
25/25 [==============================] - 0s 2ms/step - loss: 0.0167 - accuracy: 0.8850
Epoch 334/500
25/25 [==============================] - 0s 2ms/step - loss: 0.0194 - accuracy: 0.8650
Epoch 335/500
25/25 [==============================] - 0s 2ms/step - loss: 0.0175 - accuracy: 0.8800
Epoc

25/25 [==============================] - 0s 3ms/step - loss: 0.0154 - accuracy: 0.8963
Epoch 408/500
25/25 [==============================] - 0s 2ms/step - loss: 0.0178 - accuracy: 0.8813
Epoch 409/500
25/25 [==============================] - 0s 2ms/step - loss: 0.0149 - accuracy: 0.8913
Epoch 410/500
25/25 [==============================] - 0s 2ms/step - loss: 0.0142 - accuracy: 0.9075
Epoch 411/500
25/25 [==============================] - 0s 3ms/step - loss: 0.0154 - accuracy: 0.8925
Epoch 412/500
25/25 [==============================] - 0s 2ms/step - loss: 0.0147 - accuracy: 0.9013
Epoch 413/500
25/25 [==============================] - 0s 3ms/step - loss: 0.0148 - accuracy: 0.8988
Epoch 414/500
25/25 [==============================] - 0s 2ms/step - loss: 0.0163 - accuracy: 0.8925
Epoch 415/500
25/25 [==============================] - 0s 2ms/step - loss: 0.0148 - accuracy: 0.9050
Epoch 416/500
25/25 [==============================] - 0s 2ms/step - loss: 0.0147 - accuracy: 0.9100
Epoc

25/25 [==============================] - 0s 3ms/step - loss: 0.0126 - accuracy: 0.9212
Epoch 489/500
25/25 [==============================] - 0s 3ms/step - loss: 0.0129 - accuracy: 0.9187
Epoch 490/500
25/25 [==============================] - 0s 3ms/step - loss: 0.0142 - accuracy: 0.9050
Epoch 491/500
25/25 [==============================] - 0s 3ms/step - loss: 0.0141 - accuracy: 0.9112
Epoch 492/500
25/25 [==============================] - 0s 3ms/step - loss: 0.0147 - accuracy: 0.9075
Epoch 493/500
25/25 [==============================] - 0s 3ms/step - loss: 0.0120 - accuracy: 0.9212
Epoch 494/500
25/25 [==============================] - 0s 3ms/step - loss: 0.0132 - accuracy: 0.9162
Epoch 495/500
25/25 [==============================] - 0s 3ms/step - loss: 0.0129 - accuracy: 0.9100
Epoch 496/500
25/25 [==============================] - 0s 3ms/step - loss: 0.0139 - accuracy: 0.9125
Epoch 497/500
25/25 [==============================] - 0s 3ms/step - loss: 0.0130 - accuracy: 0.9062
Epoc

In [50]:
# Evaluate the model with test data
loss, accuracy = model.evaluate(X_test, y_test, verbose=2)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

model.summary()

7/7 - 0s - loss: 0.0527 - accuracy: 0.6800 - 34ms/epoch - 5ms/step
Test loss: 0.05267231538891792
Test accuracy: 0.6800000071525574
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten (Flatten)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 100)               12900     
                                                                 
 dropout (Dropout)           (None, 100)               0         
                                                                 
 dense_1 (Dense)             (None, 200)               20200     
                                                                 
 dropout_1 (Dropout)         (None, 200)               0         
                                                                 
 dense_2 (Dense)             (None, 200)               4